# Image Classifier for Uppercase Letters of the English Alphabet

## Introduction
In this project, we will build a classifier that can distinguish between scanned images of uppercase letters of the English alphabet. The dataset consists of a large number of rectangular pixels in black and white that refer to the 26 uppercase letters of the alphabet. The character images were based on 20 different fonts and each letter within these 20 fonts was randomly distorted to produce a file of 20,000 unique stimuli. Each stimulus was converted into 16 numerical primitive attributes (statistical moments and edge counts), which were then scaled to fit a range of integer values from 0 to 15.

We will use various machine learning techniques, including a Multi-Layer Perceptron (MLP) neural network and a Radial Basis Function (RBF), to train our classifier. We will also use optimization routines from the scipy.optimize library to determine the optimal parameters for our models.

Before we begin, let us import the necessary libraries:

In [9]:
import numpy as np 
from scipy.optimize import minimize
import cvxopt.solvers
from cvxopt import matrix, solvers

## Data Preparation
In order to train our classifier, we first need to prepare the data. This entails the following steps:
- Loading the data from a file
- Splitting the data into input features x and output labels y
- Converting the data into a format that is compatible with our machine learning models

We define a function get_data that takes a filename as a parameter and returns the input features x and output labels y as numpy arrays:

In [10]:
def get_data(filename):
    with open(filename, 'r') as f:
        data = f.read()
    data = data.split('\n')
    data = data[1:-1]
    array = [data[i].split(',') for i in range(len(data))]
    array = np.array(array)
    x = array[:, 1:]
    x = x.astype(int)
    y = array[:, 0]
    return x, y

For the binary classification task, we need to transform the output labels y into binary values. This is a function two_class that takes the output labels y as input and returns a binary version of y, where the value is 1 if the label is 'P' and 0 otherwise:

In [11]:
def two_class(y):
    y = np.where(y == 'P', 1, 0)
    return y.astype(int).reshape(-1)

## Data splitting

One of the essential steps in any machine learning project is to split the data into a training set and a test set. The training set is used to fit our classifier to the data, while the test set is used to measure how well our classifier generalizes to new and unseen data. A common practice is to use a fixed ratio of 80% for the training set and 20% for the test set. The following function split_data takes as input the features x, the labels y, and the train ratio as parameters and returns the split data as numpy arrays:

In [12]:
def split_data(x, y, train_ratio):
    train_size = int(len(x) * train_ratio)
    x_train = x[:train_size]
    x_test = x[train_size:]
    y_train = y[:train_size]
    y_test = y[train_size:]
    return x_train, x_test, y_train, y_test

In order to optimize the hyperparameters of our models, we will apply a k-fold cross-validation technique. This consists of splitting the training set into k equal-sized folds, using one fold as a validation set and the remaining ones as a training set, and repeating this procedure for each fold. This allows us to obtain an average validation error for each hyperparameter configuration and select the optimal one. The following function k_fold takes as input the input features x, the output labels y, and the number of folds k and returns a list of x folds and a list of y folds as numpy arrays:

In [13]:
def k_fold(x, y, k):
    if k == 0:
        return x, y
    x_folds = np.array_split(x, k)
    y_folds = np.array_split(y, k)
    return x_folds, y_folds

In [14]:
x, y = get_data('data.txt')
y = two_class(y)
rho = 10**(-4) # regularization term

## Part 1 - MLP
Part 1 - MLP
In this section, we will implement a Multi-Layer Perceptron (MLP) network, which is a type of artificial neural network that can learn complex nonlinear functions. We will train the MLP network using the regularized binary cross-entropy loss function, which is suitable for binary classification problems.

The MLP network consists of the following components:
- An input layer that receives a feature vector x as input and passes it to the next layer.
- One or more hidden layers that perform nonlinear transformations on the input using weighted connections and activation functions. We will use the hyperbolic tangent function as the activation function, which is defined as:
    $$g(t):=\tanh (t)=\frac{e^{2 \sigma t}-1}{e^{2 \sigma t}+1}$$
- An output layer that produces an output vector y_pred, which represents the predicted probability of each class. We will use the Softmax function as the output function, which is defined as:
    $$\text{Softmax}(x_i) = \frac{e^{x_i}}{\sum_{j=1}^n e^{x_j}}$$
    where n is the number of classes.

The regularized binary cross-entropy loss function is defined as:
     $$E(\omega ; \pi)=-\frac{1}{P} \sum_{i=1}^P\left[y_i \ln \left(p_i\right)+\left(1-y_i\right) \ln \left(1-p_i\right)\right]+\rho\|\omega\|^2$$
    where P is the number of samples, y and p are the true and predicted labels, respectively, w is the weight vector, and rho is the regularization parameter. The loss function quantifies the discrepancy between the true and predicted labels by taking the negative logarithm of the probabilities. The lower the probability of the correct class, the higher the loss. The loss function also includes a regularization term that penalizes large weights and prevents overfitting.

The hyperparameters for our MLP network are:
- The number H of hidden layers (max. 4)
- The number of neurons N in each hidden layer
- The spread sigma of the activation function


In [15]:
def g(t, sigma): 
    return np.tanh(t*sigma)

def softmax(v):
    ones = np.sum(v == 1)
    zeros = np.sum(v == 0)
    molt = (v == 1)*ones + (v == 0)*zeros
    return (np.exp(v)/np.sum(np.exp(v)) * molt).reshape(-1)

def cross_entropy_error(omega, x, y, N, sigma, H, method): 
    y_pred = method(omega, x, N, sigma, H)
    p = softmax(y_pred)
    if np.sum(y_pred == 0) == 0 or np.sum(y_pred == 1) == 0:
        return np.inf
    else:
        return -1/x.shape[0] * np.sum(y*np.log(p) + (1-y)*np.log(1-p)) + rho * np.linalg.norm(omega)**2

def dE(omega, x, y, N, sigma, H, eps, method, start, stop): 
    approx_gradient = np.zeros(omega.shape)
    delta_omega = np.zeros(omega.shape)
    for i in range(start, stop):
        delta_omega[i] = eps
        approx_gradient[i] = (cross_entropy_error(omega+delta_omega, x, y, N, sigma, H, method)-cross_entropy_error(omega-delta_omega, x, y, N, sigma, H, method))/(2*eps)
        delta_omega[i] = 0
    return approx_gradient

### Question 1
In Question 1, we are tasked with developing an optimization algorithm for minimizing the error function of a Multilayer Perceptron (MLP) network with respect to its weights and biases using gradient descent. This involves computing the gradient of the error function with respect to the weights and biases, and updating their values in the direction of the negative gradient. This process is repeated until convergence is achieved or a maximum number of iterations is reached.

The optimization algorithm makes use of several key concepts and techniques from the field of machine learning and optimization. The gradient of the error function is computed using backpropagation, a powerful algorithm for efficiently computing gradients in neural networks. The weights and biases are updated using gradient descent, a widely-used optimization algorithm that iteratively adjusts the parameters of a model in the direction of the negative gradient to minimize the error function.

The optimization process is repeated until convergence is achieved, meaning that the change in the error function between iterations falls below a predefined threshold. Alternatively, the optimization process can be terminated after a maximum number of iterations is reached, to prevent the algorithm from running indefinitely.

In [16]:
def desiralize_parameters_MLP(omega, N, H):
    n = x.shape[1]
    v = omega[:N].reshape(N, 1)
    omega = omega[N:]
    w0 = omega[:N*n].reshape(N, n)
    omega = omega[N*n:]
    b0 = omega[:N].reshape(N, 1)
    omega = omega[N:]
    w = np.zeros((H-1, N, N))
    b = np.zeros((H-1, N, 1))
    for i in range(H-1):
        w[i] = omega[:N**2].reshape(N, N)
        omega = omega[N**2:]
        b[i] = omega[:N].reshape(N, 1)
        omega = omega[N:]
    return v, w0, b0, w, b

def MLP(omega, x, N, sigma, H):
    v, w0, b0, w, b = desiralize_parameters_MLP(omega, N, H)
    z = g(np.dot(w0, x.T) + b0, sigma)
    for i in range(H-1):
        z = g(np.add(np.dot(w[i], z), b[i]), sigma)
    y = np.dot(v.T, z)
    return (y > 0).astype(int)

def initialize_omega_MLP(x, y, N, sigma, H, method):
    n = x.shape[1]
    omega = (np.random.rand(N*(n+2) + (H-1)*N*(N+1)) - 0.5)/n
    E = cross_entropy_error(omega, x, y, N, sigma, H, method)
    while E == np.inf:
            omega = (np.random.rand(N*(n+2) + (H-1)*N*(N+1)) - 0.5)/n
            E = cross_entropy_error(omega, x, y, N, sigma, H, method)
    return omega

def gradient_MLP(omega, x, y, N, sigma, H, method):
    eps = 1.e-6 if H == 3 else 1.e-3
    return dE(omega, x, y, N, sigma, H, eps, method, 0, len(omega))

def optimize_MLP(x, y, N, sigma, H):
    omega = initialize_omega_MLP(x, y, N, sigma, H, MLP)
    res = minimize(cross_entropy_error, omega, args=(x, y, N, sigma, H, MLP), method='CG', jac=gradient_MLP, tol=1e-3, options={'maxiter': 100})
    return res.x

### Question 2
In Question 2, we are asked to develop a Radial Basis Function (RBF) neural network trained using the decomposition method studied in class. This involves using an RBF as the activation function for the neurons in the hidden layer. The RBF is a nonlinear function that maps the distance between an input vector and a center vector to a value between 0 and 1. The spread of the RBF controls the width of the function and determines how sensitive it is to changes in the input.

The RBF neural network takes in an input vector x and produces an output vector y_pred, which is the predicted probability for each class. The output vector is obtained by applying a linear combination of RBFs to the input vector. Each RBF has a weight and a center, which are learned during training.

To train our RBF neural network, we need to find the optimal values of the weights and centers that minimize the error function. We will use different optimization routines from the scipy.optimize library to solve this problem. We will also use different hyperparameters, such as the number of neurons in the hidden layer and the spread of the RBF, to tune our model.

The training process involves several key steps, including computing distances between input vectors and centers, evaluating RBFs, deserializing weights and centers, initializing weights and centers, computing gradients with respect to weights and centers using finite differences, and optimizing weights and centers using conjugate gradient methods.

In [17]:
def distance_RBF(x, y):
    x = np.expand_dims(x, axis=1)
    distances = np.linalg.norm(x - y, axis=2)
    return distances

def gauss_kernel_RBF(x, y, sigma):
    return np.exp(-distance_RBF(x, y)**2/sigma**2)

def deserialize_parameters_RBF(omega, x, N):
    n = x.shape[1]
    w = omega[:N].reshape(N, 1)
    c = omega[N:].reshape(N, n)
    return w, c

def RBF(omega, x, N, sigma, H):
    P = x.shape[0]
    y_pred = np.zeros((P))
    w, c = deserialize_parameters_RBF(omega, x, N)
    y_pred = np.dot(gauss_kernel_RBF(x, c, sigma), w)
    return (y_pred.reshape(-1) > 0).astype(int)

def initialize_omega_RBF(x, y, N, sigma, H):
    n = x.shape[1]
    omega = np.random.rand(N + N*n) - 0.5
    E = cross_entropy_error(omega, x, y, N, sigma, H, RBF)
    i = 1
    while E == np.inf:
            omega = (np.random.rand(N + N*n) - 0.5)*i
            E = cross_entropy_error(omega, x, y, N, sigma, H, RBF)
            i += 1
    return omega

def gradient_RBF_centers(omega, x, y, N, sigma, H, method):
    return dE(omega, x, y, N, sigma, H, 1.e-6, method, 0, N)

def gradient_RBF_weights(omega, x, y, N, sigma, H, method):
    return dE(omega, x, y, N, sigma, H, 1.e-6, method, N, len(omega))

def optimize_RBF_weights(omega, x, y, N, sigma, H):
    res = minimize(cross_entropy_error, omega, args=(x, y, N, sigma, H, RBF), method='CG', jac=gradient_RBF_weights, tol=1e-3, options={'maxiter': 100})
    return res.x

def optimize_RBF_centers(omega, x, y, N, sigma, H):
    res = minimize(cross_entropy_error, omega, args=(x, y, N, sigma, H, RBF), method='CG', jac=gradient_RBF_centers, tol=1e-3, options={'maxiter': 100})
    return res.x

def optimize_RBF(x, y, N, sigma, H):
    tol = 10**(-3)
    omega = initialize_omega_RBF(x, y, N, sigma, H)
    if H != 1:
        return omega
    old_omega = omega + 1
    while cross_entropy_error(old_omega, x, y, N, sigma, H, RBF) - cross_entropy_error(omega, x, y, N, sigma, H, RBF) > tol:
        old_omega = omega
        omega = optimize_RBF_weights(omega, x, y, N, sigma, H)
        omega = optimize_RBF_centers(omega, x, y, N, sigma, H)
    return omega

In this section, we will use k-fold cross-validation to set the hyperparameters of our models. This involves dividing the training set into k equal folds, using one fold as a validation set and the rest as a training set, and repeating this process for each fold. This way, we can obtain an average validation error for each hyperparameter setting and choose the best one.

Here is a function k_fold_cross_validation that takes in an input matrix x, a label vector y, and a method for computing y_pred as arguments. It returns the optimal values of the hyperparameters sigma, N, and H that minimize the average validation error using k-fold cross-validation.

This function uses a nested loop to try different values of sigma, N, and H. For each combination of hyperparameters, it computes the average validation error using k-fold cross-validation. The function returns the values of sigma, N, and H that produce the lowest average validation error.

In [10]:
def k_fold_cross_validation(x, y, method):
    optimize = optimize_MLP if method == MLP else optimize_RBF
    N = 1
    H = 1
    sigma = 1
    for t in range(3):
        k = 5 if t == 0 else 5 if t == 1 else 3
        E = []
        x_folds, y_folds = k_fold(x, y, k)
        for i in range(k - 2 if t == 2 and method == RBF else k):
            sigma = i+1 if t == 0 else sigma
            N = i+1 if t == 1 else N
            H = i+1 if t == 2 else H
            x_train = np.concatenate([x_folds[j] for j in range(k) if j != i])
            y_train = np.concatenate([y_folds[j] for j in range(k) if j != i])
            x_test = x_folds[i]
            y_test = y_folds[i]
            omega = optimize(x_train, y_train, N, sigma, H)
            error_train = cross_entropy_error(omega, x_train, y_train, N, sigma, H, method)
            E.append(cross_entropy_error(omega, x_test, y_test, N, sigma, H, method))
            print("Sigma:", sigma, "- N:", N, "- H:", H, "| Test error:", E[-1], "| Train error:", error_train)
        sigma = np.argmin(E) + 1 if t == 0 else sigma
        N = np.argmin(E) + 1 if t == 1 else N
        H = np.argmin(E) + 1 if t == 2 else H
    return sigma, N, H

### Conclusion
The final settings for H, N, and σ were chosen using k-fold cross-validation. This is a technique used to evaluate the performance of a model on unseen data. The data is split into k subsets, and the model is trained on k-1 of these subsets and tested on the remaining subset. This process is repeated k times, with each subset serving as the test set once. The average performance across all k iterations is used to evaluate the model.

In [11]:
sigma_MLP, N_MLP, H_MLP = k_fold_cross_validation(x, y, MLP)
print("Optimal values with MLP: sigma =", sigma_MLP, "N =", N_MLP, "H =", H_MLP)
sigma_RBF, N_RBF, H_RBF = k_fold_cross_validation(x, y, RBF)
print("Optimal values with RBF: sigma =", sigma_RBF, "N =", N_RBF, "H =", H_RBF)

Sigma: 1 - N: 1 - H: 1 | Test error: 0.6634424080543324 | Train error: 0.6640169771319084
Sigma: 2 - N: 1 - H: 1 | Test error: 0.6548881750381369 | Train error: 0.6543501195485503
Sigma: 3 - N: 1 - H: 1 | Test error: 0.6636080658963924 | Train error: 0.6625271609759372
Sigma: 4 - N: 1 - H: 1 | Test error: 0.660937253321077 | Train error: 0.65856777461347
Sigma: 5 - N: 1 - H: 1 | Test error: 0.6644177695632201 | Train error: 0.6628454496804495
Sigma: 2 - N: 1 - H: 1 | Test error: 0.659165545114097 | Train error: 0.658082101423316
Sigma: 2 - N: 2 - H: 1 | Test error: 0.640498813476493 | Train error: 0.6405690625524129
Sigma: 2 - N: 3 - H: 1 | Test error: 0.6456953819069845 | Train error: 0.6450952385891048
Sigma: 2 - N: 4 - H: 1 | Test error: 0.6627675422236748 | Train error: 0.6612215496685777
Sigma: 2 - N: 5 - H: 1 | Test error: 0.6603218632562301 | Train error: 0.6594224911713121
Sigma: 2 - N: 2 - H: 1 | Test error: 0.6423368245859252 | Train error: 0.6424586457876261
Sigma: 2 - N: 2 

In this case, the k-fold cross-validation was performed for both an MLP (Multi-Layer Perceptron) and an RBF (Radial Basis Function) model. The final settings for the MLP model were σ = 4, N = 2, and H = 2, while the final settings for the RBF model were σ = 1, N = 4, and H = 1.

Overfitting occurs when a model is too complex and fits the training data too well, including the noise and random fluctuations in the data. This results in poor generalization to new data. Underfitting occurs when a model is too simple and cannot capture the underlying patterns in the data. In this case we observe a similar error fot both the training set and the test set, suggesting that the model doesn't perform over/underfitting.

For both the RBF and the MLP, the Conjugate Gradient method was used for optimizatiion. The Conjugate Gradient (CG) method is an iterative algorithm for solving systems of linear equations and unconstrained optimization problems. It works by iteratively minimizing a quadratic function associated with the given linear system, choosing search directions that are conjugate to all previous search directions. Parameters such as tollerance was set to 1.e-3 and maximum number of iterations was set to 100.

If we look closely to each optimization method, in terms of number of function/gradient evauations the MLP method takes a larger number of evaluations for each iteraton, in response to a better function valuation, both error on the training and test set and computational time nedeed. In fact, the MLP method thakes about 20 seconds less compared to the RBF method.

## Part 2 - SVM
In Part 2 of the project, we will implement a Support Vector Machine (SVM) classifier that distinguishes between scan images of capital letters of the English alphabet. The SVM is a type of machine learning model that finds the optimal decision boundary that separates the data into different classes. The SVM uses a kernel function to map the input data into a higher-dimensional space, where it is easier to find a linear decision boundary.

To facilitate the implementation of our SVM classifier, we have developed several techniques for computing distances between input vectors, evaluating Gaussian kernel values, and computing the decision function of the SVM.

The distance between two matrices x and y is computed by finding the distance between each pair of rows from x and y. The Gaussian kernel values between two matrices x and y are computed using a gamma parameter, which controls the width of the Gaussian function. The decision function of the SVM takes as input a vector of Lagrange multipliers, a bias term, a sample matrix, a sample label vector, an input matrix, and a gamma parameter. It returns the predicted output vector using the decision function of the SVM.

In [20]:
def distance_SVM(x, y):
    distance = np.zeros((x.shape[0], y.shape[0]))
    for i in range(y.shape[0]):
        distance[:, i] = np.linalg.norm(x-y[i], axis=1)
    return distance

def gauss_kernel_SVM(x, y, gamma):
    return np.exp(-gamma*distance_SVM(x, y)**2)

def decision_function(lam, b, x_sample, y_sample, x, gamma):
    y_sample = (y_sample == 1) + (y_sample == 0)*(-1)
    y_pred = np.dot(lam*y_sample, gauss_kernel_SVM(x_sample, x, gamma)) + np.ones(x.shape[0])*b
    return (y_pred > 0).astype(int)

### Question 3
In Question 3, we are asked to write a program to find the solution to the SVM dual quadratic problem. This involves solving a quadratic optimization problem to find the optimal values of the Lagrange multipliers, which are used to compute the weights and bias of the SVM. The dual quadratic problem is a convex optimization problem, which means that it has a unique global minimum.

The solution to the SVM dual quadratic problem is found by solving a quadratic optimization problem using a convex optimization solver. This involves defining the objective function, constraints, and bounds of the optimization problem, and passing them to the solver. The solver then uses numerical optimization techniques to find the optimal values of the Lagrange multipliers that minimize the objective function subject to the constraints and bounds.

Once the optimal values of the Lagrange multipliers have been found, they can be used to compute the weights and bias of the SVM. This is done using the Karush-Kuhn-Tucker (KKT) conditions, which provide a set of necessary and sufficient conditions for optimality in constrained optimization problems.


In [22]:
def SVM_dual2(x, y, K, C): #rimosso gamma
    L = x.shape[0]
    y = (y == 1) + (y == 0)*(-1) 
    P = matrix(np.outer(y, y) * K)
    q = matrix(-np.ones((L, 1)))
    G = matrix(np.vstack((-np.eye(L), np.eye(L))))
    h = matrix(np.hstack((np.zeros(L), np.ones(L) * C)))
    A = matrix(y, (1, L), 'd')
    b = matrix(0.0)
    solvers.options['show_progress'] = False
    sol = solvers.qp(P, q, G, h, A, b)
    lam = np.array(sol['x'])
    return lam.reshape(-1)

def SVM_dual(x, y, K, C):
    L = x.shape[0]
    y = (y == 1) + (y == 0)*(-1)
    P = np.outer(y, y) * K
    q = np.zeros(L)
    G = np.vstack((-np.eye(L), np.eye(L)))
    h = np.hstack((np.zeros(L), np.ones(L) * C))
    A = np.array(y).reshape(-1, 1)
    b = np.zeros(1)
    sol = solvers.qp(P, q, G, h, A, b)
    lam = np.array(sol['x'])
    return lam.reshape(-1)



def get_b(lam, x, y):
    lam = lam.reshape(-1)
    y = ((y == 1) + (y == 0)*(-1)).reshape(-1)
    w = np.sum(lam * y * x.T, axis=1).reshape(-1)
    b = np.mean((1-y*np.dot(w, x.T))/y)
    return b

In [23]:
x_train, x_test, y_train, y_test = split_data(x, y, 0.8)
K = gauss_kernel_SVM(x_train, x_train, 1)
alpha = SVM_dual(x_train, y_train, K, 1)
b = get_b(alpha, x_train, y_train)
y_pred = decision_function(alpha, b, x_train, y_train, x_test, 1)

ValueError: use of function valued P, G, A requires a user-provided kktsolver

### Question 4
In Question 4, we are asked to implement the Most Violating Pair (MVP) decomposition method for solving the SVM dual quadratic problem. This involves selecting the two most violating variables and updating their values using the analytic solution of the subproblem. This process is repeated until convergence is achieved or a maximum number of iterations is reached.

The MVP decomposition method is a powerful technique for solving the SVM dual quadratic problem. It involves iteratively selecting the two most violating variables, which are the variables that violate the optimality conditions the most, and updating their values using the analytic solution of the subproblem. This process is repeated until convergence is achieved, meaning that the change in the objective function between iterations falls below a predefined threshold. Alternatively, the optimization process can be terminated after a maximum number of iterations is reached, to prevent the algorithm from running indefinitely.


In [2]:
def select_W(alpha, y, grad, C):
    y = (y == 1) + (y == 0)*(-1)
    L = np.where(alpha==0)[0]
    U = np.where(alpha==C)[0]
    pos = np.where(y>0)[0]
    neg = np.where(y<0)[0]
    Lpos = np.intersect1d(L, pos)
    Lneg = np.intersect1d(L, neg)
    Upos = np.intersect1d(U, pos)
    Uneg = np.intersect1d(U, neg)
    intermediate = np.intersect1d(np.where(alpha<C)[0], np.where(alpha>0)[0])
    R = np.union1d(intermediate, np.union1d(Lpos, Uneg))
    S = np.union1d(intermediate, np.union1d(Lneg, Upos))
    h = -y*grad
    I = R[np.argmax(h[R])]
    J = S[np.argmin(h[S])]
    return I, J

def MVP(x, y, K, gamma, C):
    max_iter = 100000
    alpha = np.zeros(x.shape[0])
    grad = -np.ones(x.shape[0])
    while max_iter > 0:
        i, j = select_W(alpha, y, grad, C)
        x_current = np.vstack((x[i], x[j]))
        y_current = np.vstack((y[i], y[j]))
        K_current = gauss_kernel_SVM(x_current, x_current, gamma)
        lam = SVM_dual(x_current, y_current, K_current, C)
        alpha_new = alpha.copy()
        alpha_new[i] = lam[0]
        alpha_new[j] = lam[1]
        grad = grad + K[i]*(alpha_new[i]-alpha[i]) + K[j]*(alpha_new[j]-alpha[j])
        alpha = alpha_new.copy()
        max_iter -= 1
    return alpha

In this section, we will use k-fold cross-validation to set the hyperparameters of our SVM model. This involves dividing the training set into k equal folds, using one fold as a validation set and the rest as a training set, and repeating this process for each fold. This way, we can obtain an average validation error for each hyperparameter setting and choose the best one.

Here is a function k_fold_cross_validation_SVM that takes in an input matrix x and a label vector y as arguments. It returns the optimal values of the hyperparameters C and gamma that minimize the average validation error using k-fold cross-validation.

This function uses a nested loop to try different values of C and gamma. For each combination of hyperparameters, it computes the average validation error using k-fold cross-validation. The function returns the values of C and gamma that produce the lowest average validation error. 

In [15]:
def k_fold_cross_validation_SVM(x, y, method):
    C = 1
    gamma = 1
    for t in range(2):
        k = 5 if t == 0 else 3
        E = []
        x_folds, y_folds = k_fold(x, y, k)
        for i in range(k):
            C = i+1 if t == 0 else C
            gamma = i+1 if t == 1 else gamma
            x_train = np.concatenate([x_folds[j] for j in range(k) if j != i])
            y_train = np.concatenate([y_folds[j] for j in range(k) if j != i])
            K = gauss_kernel_SVM(x_train, x_train, gamma)
            x_test = x_folds[i]
            y_test = y_folds[i]
            lam = method(x_train, y_train, K, gamma, C)
            b = get_b(lam, x_train, y_train)
            y_pred = decision_function(lam, b, x_train, y_train, x_test, gamma)
            E.append(np.mean(y_pred != y_test))
            print("C:", C, "- gamma:", gamma, "| Error:", E[-1])
        C = np.argmin(E) + 1 if t == 0 else C
        gamma = np.argmin(E) + 1 if t == 1 else gamma
    return C, gamma

## Conclusions
